In [45]:
import os, sys, shutil
import random
import numpy as np
import json

In [9]:
sys.path.append("../")
%load_ext autoreload
%autoreload 2

In [71]:
from utils.utils import makeMini
from scorer_scripts_v2.scorer_scripts_v2.score_detections import score

In [72]:
os.getcwd()

'C:\\Users\\wsutt\\Desktop\\alphaPilot\\books'

##### Make Mini Img-Set / Copy Files

In [74]:
makeMini(N=10)

../mini/mini1
10
['IMG_0160.JPG', 'IMG_0696.JPG', 'IMG_3751.JPG']...
output truth  truth.json
